# Initialize Map


In [1]:
# map_init = False

# while not map_init:

    # map_occ_grid , map_info = get map with getMap service from nav_msgs 

# if range_method = GLT:
#   compute a LUT from the map before starting with pf 
#   check if LUT computation is time intensive, otherwise use precomputed LUT from numpy file  

In [ ]:
def get_omap(self):
        '''
        Fetch the occupancy grid map from the map_server instance, and initialize the correct
        RangeLibc method. Also stores a matrix which indicates the permissible region of the map
        '''
        rospy.wait_for_service("static_map")
        map_msg = rospy.ServiceProxy("static_map", GetMap)().map

        self.map_info = map_msg.info
        oMap = range_libc.PyOMap(map_msg)
        self.MAX_RANGE_PX = int(
            self.MAX_RANGE_METERS / self.map_info.resolution)

        # initialize range method
        rospy.loginfo(f"Initializing range method: {self.WHICH_RM}")
        if self.WHICH_RM == "bl":
            self.range_method = range_libc.PyBresenhamsLine(
                oMap, self.MAX_RANGE_PX)
        elif "cddt" in self.WHICH_RM:
            self.range_method = range_libc.PyCDDTCast(
                oMap, self.MAX_RANGE_PX, self.THETA_DISCRETIZATION)
            if self.WHICH_RM == "pcddt":
                rospy.loginfo("Pruning...")
                self.range_method.prune()
        elif self.WHICH_RM == "rm":
            self.range_method = range_libc.PyRayMarching(
                oMap, self.MAX_RANGE_PX)
        elif self.WHICH_RM == "rmgpu":
            self.range_method = range_libc.PyRayMarchingGPU(
                oMap, self.MAX_RANGE_PX)
        elif self.WHICH_RM == "glt":
            self.range_method = range_libc.PyGiantLUTCast(
                oMap, self.MAX_RANGE_PX, self.THETA_DISCRETIZATION)
        rospy.loginfo("Done loading map")

        # 0: permissible, -1: unmapped, 100: blocked
        array_255 = np.array(map_msg.data).reshape(
            (map_msg.info.height, map_msg.info.width))

        # 0: not permissible, 1: permissible
        self.permissible_region = np.zeros_like(array_255, dtype=bool)
        self.permissible_region[array_255 == 0] = 1

        # // Sanity Check
        # _, axs = plt.subplots(nrows=2, ncols=1)
        # axs[0].set_title("Original data")
        # axs[0].imshow(array_255)
        # axs[1].set_title("Permissible Region")
        # im=axs[1].imshow(self.permissible_region)
        # plt.colorbar(im, orientation="horizontal")
        # plt.show()

        self.map_initialized = True